In [1]:
# first of all we need to import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import datetime as dt

pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

from ydata_profiling import ProfileReport
from google.cloud import bigquery

ModuleNotFoundError: No module named 'google.cloud'

In [ ]:
# Then we create a BigQuery client object to pull the data into the workbench environment, and just like in the BigQuery environment, we pull our data with SQL commands and bring them into a dataframe format.
client = bigquery.Client()

raw_data = client.query("SELECT * FROM `bqml-cltv.cltv_dataset.flo_data_20k`").to_dataframe()
df = raw_data.copy()
raw_data.head()

In [ ]:
# we have about 20.000 row and 12 feature 
print("shape of the original data", df.shape)
df.head()

In [ ]:
# we received the data from BigQuery, so the date columns seem to be of the wrong type, we have to fix them.
df.dtypes

In [ ]:
# get column names as a list
date_columns = list(["first_order_date","last_order_date","last_order_date_online","last_order_date_offline"])

for column in date_columns:
    df[column] = pd.to_datetime(df[column])

In [ ]:
# profiling
profile = ProfileReport(df, title="FLO Profiling Report")
profile.to_file("FLO_Profiling_Report.html")

In [ ]:
# in order to observe the outlier values in the columns containing the number of orders and order amounts in both online and offline channels, let's simply visualize these columns with the boxplot.
order_columns = ["order_num_total_ever_online","order_num_total_ever_offline"]
value_columns = ["customer_value_total_ever_online","customer_value_total_ever_offline"]

In [ ]:
for x in order_columns:
    plt.figure(figsize=(12, 4))
    sns.boxplot(df[x], orient="h")
    plt.xlabel(x)
    plt.show()

In [ ]:
for x in value_columns:
    plt.figure(figsize=(12, 4))
    sns.boxplot(df[x], orient="h")
    plt.xlabel(x)
    plt.show()

In [ ]:
# let's write a simple function to see the missing values, this function will return the number of missing values and percentage of missing values.
def missing_data_report(data):
    # calculate the number of missing values
    missing_values = data.isnull().sum()

    # calculate the percentage of the missing values
    percent_missing = (missing_values / data.shape[0]) * 100

    # merge the results into a DataFrame.
    result = pd.DataFrame({
        "number of missing values": missing_values,
        "percentage of missing values": percent_missing
    })

    # sort the results by number of missing values
    result = result.sort_values(by="number of missing values", ascending=False)

    return result

missing_data_report(df)

In [ ]:
# As you can see we have no missing values, this is a good news because each observation is important for us, now we will define the numeric variables and save it as a csv file.
df.describe().T.to_csv("summary_of_data.csv")

In [ ]:
# There are different methods to identify outliers, in this project I will use the IQR method. Let's see these outliers, I have written a simple function to see outliers, in this function we need to write the values Q1 and Q3.
def detect_outliers(data, q1 = 0.25, q3 = 0.75, noe = 5):
    outliers = {}
    for column in data.columns:
        if pd.api.types.is_numeric_dtype(data[column]):
            Q1 = data[column].quantile(q1)
            Q3 = data[column].quantile(q3)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            outlier_indices = (data[column] < lower_bound) | (data[column] > upper_bound)
            outliers[column] = data[column][outlier_indices]
            
    return pd.DataFrame(outliers).head(noe)

In [ ]:
detect_outliers(df, q1 = 0.01, q3 = 0.99, noe = 10)

In [ ]:
# Now that we have seen our outliers, we can either discard these outliers from the dataset or suppress them according to the values we want, we said that each observation is important, so we will suppress the observations that contain these outliers.
def outlier_thresholds(data, variable, q1 = 0.25, q3 = 0.75):
    Q1 = data[variable].quantile(q1)
    Q3 = data[variable].quantile(q3)
    interquantile_range = Q3 - Q1
    up_limit = Q1 + 1.5 * interquantile_range
    low_limit = Q3 - 1.5 * interquantile_range
    return low_limit, up_limit

 
def replace_with_thresholds(data, variable, q1 = 0.25, q3 = 0.75):
    low_limit, up_limit = outlier_thresholds(data, variable, q1, q3)
    data.loc[(data[variable] < low_limit), variable] = low_limit.round()
    data.loc[(data[variable] > up_limit), variable] = up_limit.round()

In [ ]:
clean_data = df.copy()

In [ ]:
replace_with_thresholds(clean_data, "order_num_total_ever_online", q1 = 0.01, q3 = 0.99)
replace_with_thresholds(clean_data, "order_num_total_ever_offline", q1 = 0.01, q3 = 0.99)
replace_with_thresholds(clean_data, "customer_value_total_ever_offline", q1 = 0.01, q3 = 0.99)
replace_with_thresholds(clean_data, "customer_value_total_ever_online", q1 = 0.01, q3 = 0.99)

In [ ]:
detect_outliers(clean_data, q1 = 0.01, q3 = 0.99)

# creating metrics to calculate CLTV

In [ ]:
# We use Vertex Ai Workbench for these calculations because of the high computing power capacity offered by Google. 

In [ ]:
# Create a new more simple dataframe to calculate the cltv easily
clv_data = clean_data[["master_id",
                      "first_order_date",
                      "last_order_date",
                      "order_num_total_ever_online",
                      "order_num_total_ever_offline",
                      "customer_value_total_ever_offline",
                      "customer_value_total_ever_online"]]

In [ ]:
clean_data.to_csv("clean_data_for_cltv.csv")

In [ ]:
# We set the analysis date as 2 days after the last date in the dataset for this demo.
last_day_on_data = clv_data.last_order_date.max()
analysis_day = last_day_on_data + dt.timedelta(days=2)

a_week = 7

In [1]:
# Tenure = Analysis day - Customer's first order date / 7. Tenure metric should be weekly.

clv_data["T_weekly"] = analysis_day - clv_data["first_order_date"]
clv_data["T_weekly"] = ((clv_data["T_weekly"] // pd.Timedelta(days=1)) / a_week).round()

NameError: name 'analysis_day' is not defined

In [ ]:
# Recency = Customer's last order date - Customer's first order date / 7. Recency value should be daily, weekly or yearly but we get it weekly.

clv_data["recency_weekly"] = clv_data["last_order_date"] - clv_data["first_order_date"]
clv_data["recency_weekly"] = ((clv_data["recency_weekly"] // pd.Timedelta(days=1)) / a_week).round()

In [ ]:
# Frequency = Total number of transaction. Minimum frequency should be 2 to calculate the CLTV.

clv_data["frequency"] = (clv_data["order_num_total_ever_online"] + clv_data["order_num_total_ever_offline"]).round()
clv_data = clv_data[clv_data["frequency"] > 2]

In [ ]:
# Monetary = Total purchase amount / Total number of transaction.

clv_data["monetary_avg"] = ((clv_data["customer_value_total_ever_offline"] + clv_data["customer_value_total_ever_online"]) / clv_data["frequency"]).round()

In [ ]:
clv_data.head()

In [ ]:
# bg/nbd model
from lifetimes import BetaGeoFitter

# fit the model
bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(clv_data["frequency"],
        clv_data["recency_weekly"],
        clv_data["T_weekly"])

# predict for 3 months
three_month = 4 * 3
clv_data["exp_sales_3_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(three_month,
                                                                                        clv_data["frequency"],
                                                                                        clv_data["recency_weekly"],
                                                                                        clv_data["T_weekly"])

# predict for 6 months
six_months = 4 * 6
clv_data["exp_sales_6_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(six_months,
                                                                                        clv_data["frequency"],
                                                                                        clv_data["recency_weekly"],
                                                                                        clv_data["T_weekly"])

In [ ]:
# gamma gamma model
from lifetimes import GammaGammaFitter

# fit the model
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(clv_data['frequency'], clv_data['monetary_avg'])

# get predicts
clv_data["exp_average_value"] = ggf.conditional_expected_average_profit(clv_data['frequency'],
                                                                       clv_data['monetary_avg'])

In [ ]:
# calculate the customer life time value

clv_data["clv"] = ggf.customer_lifetime_value(bgf,
                                   clv_data['frequency'],
                                   clv_data['recency_weekly'],
                                   clv_data['T_weekly'],
                                   clv_data['monetary_avg'],
                                   time=6,  # 6 months
                                   freq="W",  # W = Week
                                   discount_rate=0.01)

In [ ]:
clv_data["customer_segment"] = pd.qcut(clv_data["clv"], 4, labels=["D", "C", "B", "A"])

In [ ]:
clv_data.head()

In [ ]:
clv_data.groupby("customer_segment").describe()

In [ ]:
segment_A_customers = clv_data[clv_data["customer_segment"] == "A"]

In [ ]:
segment_A_customers.to_csv("segment_A_customer.csv")